In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sjögrens_Syndrome/GSE66795'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A transcriptional signature of fatigue derived from patients with primary Sjögren's Syndrome"
!Series_summary	"Fatigue is a debilitating condition with a significant impact on patients’ quality of life. Fatigue is frequently reported by patients suffering from primary Sjo ̈gren’s Syndrome (pSS), a chronic autoimmune condition characterised by dryness of the eyes and the mouth. However, although fatigue is common in pSS, it does not manifest in all sufferers, providing an excellent model with which to explore the potential underpinning biological mechanisms."
!Series_overall_design	"Whole blood samples from 131 fully-phenotyped pSS patients, stratified for the presence of fatigue, collected by the UK primary Sj ̈ogren’s Syndrome Registry were used for whole genome microarray. The resulting data were analysed both on a gene by gene basis and using pre-defined groups of genes. Finally, gene set enrichment analysis (GSEA) was used as a feature selecti

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check for gene expression data availability
is_gene_available = True  # Assuming gene expression data is available in the dataset

# Step 2.1: Variable Availability
# Since there is no clear mention of Sjögren's Syndrome status being provided in the sample characteristics
# We assume the patient group (Control, Patient) refers to the Sjögren's Syndrome status.
trait_row = 2
age_row = None  # Age information is not provided in the sample characteristics
gender_row = 3  # Gender information is available in the sample characteristics

# Step 2.3: Data Type Conversion
def convert_trait(value: str) -> int:
    val = value.split(':')[-1].strip().lower()
    if val == 'control':
        return 0
    elif val == 'patient':
        return 1
    return None

def convert_age(value: str):
    return None  # Age data is not available

def convert_gender(value: str) -> int:
    val = value.split(':')[-1].strip().lower()
    if val == 'female':
        return 0
    elif val == 'male':
        return 1
    return None

# Step 3: Saving cohort information 
save_cohort_info('GSE66795', './preprocessed/Sjögrens_Syndrome/cohort_info.json', is_gene_available, trait_row is not None)

# Step 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sjögrens_Syndrome', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sjögrens_Syndrome/trait_data/GSE66795.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1632237': [0, 0], 'GSM1632238': [0, 0], 'GSM1632239': [1, 0], 'GSM1632240': [0, 0], 'GSM1632241': [1, 0], 'GSM1632242': [0, 0], 'GSM1632243': [1, 0], 'GSM1632244': [0, 0], 'GSM1632245': [1, 0], 'GSM1632246': [0, 0], 'GSM1632247': [1, 0], 'GSM1632248': [1, 0], 'GSM1632249': [0, 0], 'GSM1632250': [1, 0], 'GSM1632251': [1, 0], 'GSM1632252': [1, 0], 'GSM1632253': [1, 0], 'GSM1632254': [1, 0], 'GSM1632255': [0, 0], 'GSM1632256': [1, 0], 'GSM1632257': [1, 0], 'GSM1632258': [1, 0], 'GSM1632259': [1, 0], 'GSM1632260': [1, 0], 'GSM1632261': [1, 0], 'GSM1632262': [1, 0], 'GSM1632263': [1, 0], 'GSM1632264': [1, 0], 'GSM1632265': [0, 0], 'GSM1632266': [1, 0], 'GSM1632267': [1, 0], 'GSM1632268': [1, 0], 'GSM1632269': [0, 0], 'GSM1632270': [1, 0], 'GSM1632271': [0, 0], 'GSM1632272': [0, 0], 'GSM1632273': [1, 0], 'GSM1632274': [1, 0], 'GSM1632275': [1, 0], 'GSM1632276': [1, 0], 'GSM1632277': [1, 0], 'GSM1632278': [1, 0], 'GSM1632279': [1, 0], 'GSM1632280': [0, 0], 'GSM1632281': [1, 0], 'GSM16322

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:

# 1. Define the keys for identifier and gene symbol based on the previews
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Sjögrens_Syndrome/gene_data/GSE66795.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Sjögrens_Syndrome')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE66795', './preprocessed/Sjögrens_Syndrome/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Sjögrens_Syndrome/GSE66795.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Sjögrens_Syndrome', the least common label is '0.0' with 29 occurrences. This represents 18.12% of the dataset.
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 160 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.

